In [1]:
!pip install gdown --quiet

!gdown 1ZdhRqYv-JizWV6DxO6C4R_k1kxPhmlF2 -O groceries.csv
import pandas as pd
import pandas as pd
import numpy as np
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

Downloading...
From: https://drive.google.com/uc?id=1ZdhRqYv-JizWV6DxO6C4R_k1kxPhmlF2
To: /content/groceries.csv
100% 14.6k/14.6k [00:00<00:00, 20.3MB/s]


In [2]:
df = pd.read_csv("groceries.csv")
print("Shape:", df.shape)
print(df.head())


Shape: (440, 8)
   Region  Fresh  Milk  Grocery  Frozen  Detergents_Paper  Delicassen  class
0       3  12669  9656     7561     214              2674        1338      2
1       3   7057  9810     9568    1762              3293        1776      2
2       3   6353  8808     7684    2405              3516        7844      2
3       3  13265  1196     4221    6404               507        1788      1
4       3  22615  5410     7198    3915              1777        5185      1


In [15]:
print(df.info())
print(df.isnull().sum())
print(df.describe())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 440 entries, 0 to 439
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype
---  ------            --------------  -----
 0   Region            440 non-null    int64
 1   Fresh             440 non-null    int64
 2   Milk              440 non-null    int64
 3   Grocery           440 non-null    int64
 4   Frozen            440 non-null    int64
 5   Detergents_Paper  440 non-null    int64
 6   Delicassen        440 non-null    int64
 7   class             440 non-null    int64
dtypes: int64(8)
memory usage: 27.6 KB
None
Region              0
Fresh               0
Milk                0
Grocery             0
Frozen              0
Detergents_Paper    0
Delicassen          0
class               0
dtype: int64
           Region          Fresh          Milk       Grocery        Frozen  \
count  440.000000     440.000000    440.000000    440.000000    440.000000   
mean     2.543182   12000.297727   5796.265909   7951.277

In [4]:
df_filled = df.fillna(df.mean())


In [17]:
categorical_cols = df_filled.select_dtypes(include=['object', 'category']).columns.tolist()
if categorical_cols:
    df_encoded = pd.get_dummies(df_filled, columns=categorical_cols)
else:
    df_encoded = df_filled.copy()
print(df_encoded.head())

   Region  Fresh  Milk  Grocery  Frozen  Detergents_Paper  Delicassen  class
0       3  12669  9656     7561     214              2674        1338      2
1       3   7057  9810     9568    1762              3293        1776      2
2       3   6353  8808     7684    2405              3516        7844      2
3       3  13265  1196     4221    6404               507        1788      1
4       3  22615  5410     7198    3915              1777        5185      1


In [6]:
X = df_filled.drop('class', axis=1)
y = df_filled['class']

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print('Shapes:', X_train.shape, X_test.shape, y_train.shape, y_test.shape)

Shapes: (352, 7) (88, 7) (352,) (88,)


In [7]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)

X_test_scaled = scaler.transform(X_test)

In [8]:
def euclidean_distance(x1, x2):
    return np.sqrt(np.sum((x1 - x2) ** 2))

In [9]:
class KNNScratch:
    def __init__(self, k=3):
        self.k = k

    def fit(self, X_train, y_train):
        self.X_train = X_train
        self.y_train = y_train

    def predict(self, X_test):
        y_pred = [self._predict_single(x) for x in X_test]
        return np.array(y_pred)

    def _predict_single(self, x):
        distances = [euclidean_distance(x, x_train) for x_train in self.X_train]
        k_indices = np.argsort(distances)[:self.k]
        k_nearest_labels = [self.y_train[i] for i in k_indices]
        most_common = Counter(k_nearest_labels).most_common(1)
        return most_common[0][0]

In [19]:
knn = KNNScratch(k=3)
knn.fit(X_train_scaled, y_train.values)


In [20]:
y_pred = knn.predict(X_test_scaled)
print('Predictions:', y_pred[:10])


Predictions: [2 3 2 1 3 3 3 2 3 3]


In [21]:
accuracy = accuracy_score(y_test, y_pred)
print(f'KNN Scratch Model Accuracy: {accuracy:.4f}')

KNN Scratch Model Accuracy: 0.8636


In [22]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_train_scaled, y_train)

y_pred = knn.predict(X_test_scaled)

accuracy = accuracy_score(y_test, y_pred)
print(f'KNN Sklearn Model Accuracy: {accuracy:.4f}')


KNN Sklearn Model Accuracy: 0.8750
